In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Estadísticos
# ------------------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

/home/cassia_d/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/cassia_d/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Pair Programming ANOVA

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [2]:
df = pd.read_csv('datos/soci_econ_country_profiles_v5.csv', index_col=0)
df.head(2)

,country,Region,GDP per capita (current US$),Employment: Agriculture (% of employed),Employment: Services (% of employed),Agricultural production index (2004-2006=100),Urban population (% of total population)_x,Urban population growth rate (average annual %),"Fertility rate, total (live births per woman)",Infant mortality rate (per 1000 live births,...,Human capital index (HCI) (scale 0-1),"Inflation, consumer prices (annual %)","Life expectancy at birth, total (years)",Labour force participation (male pop. %),"Pop. using improved drinking water (urban, %)","Pop. using improved drinking water (rural, %)","Population age distribution (0-14, %)","Population age distribution (60+ years, %)",Education: Secondary gross enrol. ratio (per 100 pop.),Education: Tertiary gross enrol. ratio (per 100 pop.)
0,Argentina,SouthAmerica,-0.431499,-0.793293,0.635685,0.451515,1.176545,-0.146278,0.816818,0.390527,...,-0.661735,-0.179334,-0.356054,0.526225,0.213117,0.576120,0.983668,-0.481642,0.119029,0.928674
1,Australia,Oceania,1.358804,-0.729591,0.874487,-0.051041,1.039907,0.271660,0.093270,-0.510644,...,1.061166,-0.270570,1.069555,0.052782,0.675804,0.681506,-0.044663,0.256053,1.643795,1.272043


In [3]:
df.rename(columns = {col : col.replace(' ', '_') for col in df.columns}, inplace=True)

In [4]:
diccionario = {"country": "country",
                "Region": "Region",
                "GDP_per_capita_(current_US$)" : "GDP_per_capita",
                "Employment:_Agriculture_(%_of_employed)" : "Employment_Agriculture",
                "Employment:_Services_(%_of_employed)" : "Employment_Services",
                "Agricultural_production_index_(2004-2006=100)" : "Agricultural_production_index",
                "Urban_population_(%_of_total_population)_x" : "Urban_population",
                "Urban_population_growth_rate_(average_annual_%)" : "Urban_population_growth_rate",
                "Fertility_rate,_total_(live_births_per_woman)" : "Fertility_rate_total",
                "Infant_mortality_rate_(per_1000_live_births" : "Infant_mortality_rate",
                "Seats_held_by_women_in_national_parliaments_%" : "Seats_held_by_women_in_national_parliaments",
                "Individuals_using_the_Internet_(per_100_inhabitants)" : "Individuals_using_the_Internet",
                "Energy_production,_primary_(Petajoules)" : "Energy_production_primary",
                "Energy_supply_per_capita_(Gigajoules)" : "Energy_supply_per_capita",
                "Quality_Of_Life_Index" : "Quality_Of_Life_Index",
                "Purchasing_Power_Index" : "Purchasing_Power_Index",
                "Safety_Index" : "Safety_Index",
                "Health_Care_Index" : "Health_Care_Index",
                "Property_price_to_income_ratio" : "Property_price_to_income_ratio",
                "Traffic_commute_time_index" : "Traffic_commute_time_index",
                "Pollution_index" : "Pollution_index",
                "Climate_index" : "Climate_index",
                "Affordability_Index" : "Affordability_Index",
                "Cost_Of_Living_Index" : "Cost_Of_Living_Index",
                "Consumer_price_index_(2010_=_100)" : "Consumer_price_index",
                "Current_health_expenditure_(%_of_GDP)" : "Current_health_expenditure",
                "Human_capital_index_(HCI)_(scale_0-1)" : "Human_capital_index",
                "Inflation,_consumer_prices_(annual_%)" : "Inflation_consumer_prices",
                "Life_expectancy_at_birth,_total_(years)" : "Life_expectancy_at_birth_total",
                "Labour_force_participation_(male_pop._%)" : "Labour_force_participation",
                "Pop._using_improved_drinking_water_(urban,_%)" : "Pop_using_improved_drinking_water_urban",
                "Pop._using_improved_drinking_water_(rural,_%)" : "Pop_using_improved_drinking_water_rural",
                "Population_age_distribution_(0-14,_%)" : "Population_age_distribution_0_14",
                "Population_age_distribution_(60+_years,_%)" : "Population_age_distribution_60_mas",
                "Education:_Secondary_gross_enrol._ratio_(per_100_pop.)" : "Education_Secondary_gross_enrol_ratio",
                "Education:_Tertiary_gross_enrol._ratio_(per_100_pop.)" : "Education_Tertiary_gross_enrol_ratio"}


In [5]:
df.rename(columns = diccionario, inplace=True)

In [6]:
df.head(2)

,country,Region,GDP_per_capita,Employment_Agriculture,Employment_Services,Agricultural_production_index,Urban_population,Urban_population_growth_rate,Fertility_rate_total,Infant_mortality_rate,...,Human_capital_index,Inflation_consumer_prices,Life_expectancy_at_birth_total,Labour_force_participation,Pop_using_improved_drinking_water_urban,Pop_using_improved_drinking_water_rural,Population_age_distribution_0_14,Population_age_distribution_60_mas,Education_Secondary_gross_enrol_ratio,Education_Tertiary_gross_enrol_ratio
0,Argentina,SouthAmerica,-0.431499,-0.793293,0.635685,0.451515,1.176545,-0.146278,0.816818,0.390527,...,-0.661735,-0.179334,-0.356054,0.526225,0.213117,0.576120,0.983668,-0.481642,0.119029,0.928674
1,Australia,Oceania,1.358804,-0.729591,0.874487,-0.051041,1.039907,0.271660,0.093270,-0.510644,...,1.061166,-0.270570,1.069555,0.052782,0.675804,0.681506,-0.044663,0.256053,1.643795,1.272043


In [7]:
df.isnull().sum()

country                                        0
Region                                         0
GDP_per_capita                                 0
Employment_Agriculture                         0
Employment_Services                            0
Agricultural_production_index                  0
Urban_population                               0
Urban_population_growth_rate                   0
Fertility_rate_total                           0
Infant_mortality_rate                          0
Seats_held_by_women_in_national_parliaments    0
Individuals_using_the_Internet                 0
Energy_production_primary                      0
Energy_supply_per_capita                       0
Quality_Of_Life_Index                          0
Purchasing_Power_Index                         0
Safety_Index                                   0
Health_Care_Index                              0
Property_price_to_income_ratio                 0
Traffic_commute_time_index                     0
Pollution_index     

In [8]:
lm = ols('''Quality_Of_Life_Index ~ Region
            + GDP_per_capita
            + Employment_Agriculture
            + Employment_Services
            + Agricultural_production_index
            + Urban_population
            + Urban_population_growth_rate
            + Fertility_rate_total
            + Infant_mortality_rate
            + Seats_held_by_women_in_national_parliaments
            + Individuals_using_the_Internet
            + Energy_production_primary
            + Energy_supply_per_capita
            + Purchasing_Power_Index
            + Safety_Index
            + Health_Care_Index
            + Property_price_to_income_ratio
            + Traffic_commute_time_index
            + Pollution_index
            + Climate_index
            + Affordability_Index
            + Cost_Of_Living_Index
            + Consumer_price_index
            + Current_health_expenditure
            + Human_capital_index
            + Inflation_consumer_prices
            + Life_expectancy_at_birth_total
            + Labour_force_participation
            + Pop_using_improved_drinking_water_urban
            + Pop_using_improved_drinking_water_rural
            + Population_age_distribution_0_14
            + Population_age_distribution_60_mas
            + Education_Secondary_gross_enrol_ratio
            + Education_Tertiary_gross_enrol_ratio''', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
Region,13.0,73145.658560,5626.589120,1.690034e+08,2.869203e-66
GDP_per_capita,1.0,3739.835272,3739.835272,1.123318e+08,2.041146e-59
Employment_Agriculture,1.0,1673.448126,1673.448126,5.026464e+07,1.898521e-56
Employment_Services,1.0,145.412480,145.412480,4.367692e+06,1.981494e-47
Agricultural_production_index,1.0,1859.787243,1859.787243,5.586163e+07,7.738946e-57
Urban_population,1.0,357.151714,357.151714,1.072761e+07,9.546998e-51
Urban_population_growth_rate,1.0,604.780719,604.780719,1.816554e+07,1.085263e-52
Fertility_rate_total,1.0,149.827917,149.827917,4.500317e+06,1.536629e-47
Infant_mortality_rate,1.0,383.744617,383.744617,1.152637e+07,5.185086e-51
Seats_held_by_women_in_national_parliaments,1.0,1098.769654,1098.769654,3.300327e+07,6.782851e-55


In [9]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     Quality_Of_Life_Index   R-squared:                       1.000
Model:                               OLS   Adj. R-squared:                  1.000
Method:                    Least Squares   F-statistic:                 6.218e+07
Date:                   Thu, 11 May 2023   Prob (F-statistic):           1.32e-63
Time:                           09:47:08   Log-Likelihood:                 281.53
No. Observations:                     64   AIC:                            -469.1
Df Residuals:                         17   BIC:                            -367.6
Df Model:                             46                                         
Covariance Type:               nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                     136.7930      0.011   1.27e+04      0.000     136.770     136.816
Region[T.EasternAsia]                           0.0101      0.011      0.912      0.375      -0.013       0.034
Region[T.EasternEurope]                         0.0213      0.012      1.716      0.104      -0.005       0.048
Region[T.NorthernAfrica]                        0.0028      0.035      0.081      0.936      -0.070       0.076
Region[T.NorthernAmerica]                       0.0139      0.015      0.929      0.366      -0.018       0.045
Region[T.NorthernEurope]                        0.0159      0.014      1.174      0.257      -0.013       0.045
Region[T.Oceania]                               0.0127      0.012      1.078      0.296      -0.012       0.037
Region[T.South-easternAsia]                     0.0087      0.011      0.776      0.449      -0.015       0.032
Region[T.SouthAmerica]                          0.0076      0.011      0.659      0.519      -0.017       0.032
Region[T.SouthernAfrica]                        0.0145      0.028      0.518      0.611      -0.045       0.074
Region[T.SouthernAsia]                         -0.0050      0.016     -0.321      0.752      -0.038       0.028
Region[T.SouthernEurope]                        0.0187      0.012      1.539      0.142      -0.007       0.044
Region[T.WesternAsia]                           0.0068      0.013      0.518      0.611      -0.021       0.034
Region[T.WesternEurope]                         0.0108      0.013      0.828      0.419      -0.017       0.038
GDP_per_capita                                 -0.0005      0.005     -0.108      0.915      -0.010       0.009
Employment_Agriculture                          0.0029      0.005      0.543      0.594      -0.008       0.014
Employment_Services                            -0.0007      0.005     -0.134      0.895      -0.012       0.010
Agricultural_production_index                  -0.0024      0.002     -1.315      0.206      -0.006       0.001
Urban_population                                0.0025      0.004      0.641      0.530      -0.006       0.011
Urban_population_growth_rate                -8.875e-05      0.003     -0.032      0.975      -0.006       0.006
Fertility_rate_total                           -0.0060      0.006     -1.073      0.298      -0.018       0.006
Infant_mortality_rate                           0.0004      0.004      0.094      0.926      -0.008       0.009
Seats_held_by_women_in_national_parliaments    -0.0018      0.002     -0.883      0.390      -0.006       0.002
Individuals_using_the_Internet                  0.0006      0.002      0.279      0.783      -0.004       0.005
Energy_production_primary 

In [10]:
df.to_csv('datos/soci_econ_country_profiles_v6.csv')